read input pdb and xml files 

In [1]:
import sys
import jax.numpy as jnp
import openmm.app as app
import openmm.unit as unit
from dmff.api import Hamiltonian

H = Hamiltonian('forcefield.xml')
app.Topology.loadBondDefinitions("residues.xml")
pdb = app.PDBFile("water_dimer.pdb")
rc = 4

2022-06-13 18:58:04.979758: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /share/apps/Anaconda/anaconda3/lib:/usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib
2022-06-13 18:58:04.979797: W external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


generator generates parameters and stores
create potential function of each composition of energy

In [2]:
from dmff.admp.pairwise import generate_pairwise_interaction,TT_damping_qq_c6_kernel

disp_generator, pme_generator = H.getGenerators()
pot_disp, pot_pme = H.createPotential(pdb.topology, nonbondedCutoff=rc*unit.angstrom, step_pol=5)
pot_sr = generate_pairwise_interaction(TT_damping_qq_c6_kernel, disp_generator.disp_pme_force.covalent_map, static_args={})
    

prepare positions and box

In [4]:
positions = jnp.array(pdb.positions._value) * 10
a, b, c = pdb.topology.getPeriodicBoxVectors()
box = jnp.array([a._value, b._value, c._value]) * 10

consturct neighbourlist 

In [6]:
from jax_md import space, partition

displacement_fn, shift_fn = space.periodic_general(box, fractional_coordinates=False)
neighbor_list_fn = partition.neighbor_list(displacement_fn, box, rc, 0, format=partition.OrderedSparse)
nbr = neighbor_list_fn.allocate(positions)
pairs = nbr.idx.T

wrap the computation of fluctuated leading terms through a linear model realized by function compute_leading_terms in admp_calculator

In [8]:
from run import compute_leading_terms

c0, c6_list = compute_leading_terms(positions,box) # compute fluctuated leading terms
Q_local = pme_generator.params["Q_local"][pme_generator.map_atomtype]
Q_local = Q_local.at[:,0].set(c0)

initialize the calculator, then use it to get total interaction energy and force

In [ ]:
from run import *

admp_calc = generate_calculator(pot_disp, pot_pme, pot_sr, disp_generator, pme_generator)
tot_ene, tot_force = admp_calc(positions, box, pairs)
